<a href="https://colab.research.google.com/github/domingues100/Kaggle---Codes/blob/master/Trying_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e5:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73278%2F8121328%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240517%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240517T015217Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3d8b3369b3776930ebd62725f4e8606d64111f220ffbaef1bf3e46c15b32409de6ba125d569a5266a4c770b3729e7061a49e2b4cb9f8e6ae27b6c658a8e959bf7c898c6dc16538f2b1389f07a6c1676a56076bd84e244bdcdfbc0c8d6dfac179199aa4cc008ef396951c55268d912f62296cc87b48e5c240c01cee268b1ed97e2c6b4fb51d0b4186fa96907671f5a3cc497c0e6f4ef7a912456b7a79404400f962da344b3ec271d82b19f7012f8a1e1a5218ec85a925be4d715e06c2d21188e561ebbc47383cd8a45786cce40445c0daee3ab0d3e28650c77daa56bc879375338dae984f67d19c4c6d6f2028d918a90304e02b48d836f2f4aae2a862dfb1d3bb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [ ]:
!pip install -U lightautoml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.4/416.4 kB 6.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 65.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 62.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 35.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 32.9 MB/s eta 0:00:00:00:0100:01
   ━━

In [ ]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco
from sklearn.metrics import r2_score

In [ ]:
train_csv = pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test_csv = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

In [ ]:
train_csv.head()

/opt/conda/lib/python3.10/site-packages/IPython/core/formatters.py:344: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/conda/lib/python3.10/site-packages/IPython/core/formatters.py:344: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,id,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,0,5,8,5,8,6,4,4,3,3,...,5,3,3,5,4,7,5,7,3,0.445
1,1,6,7,4,4,8,8,3,5,4,...,7,2,0,3,5,3,3,4,3,0.450
2,2,6,5,6,7,3,7,1,5,4,...,7,3,7,5,6,8,2,3,3,0.530
3,3,3,4,6,5,4,8,4,7,6,...,2,4,7,4,4,6,5,7,5,0.535
4,4,5,3,2,6,4,4,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [ ]:
Y = train_csv["FloodProbability"]
train_csv.drop(columns = ["FloodProbability", "id"], inplace = True)

test_csv = pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")
ids = test_csv["id"]
test_csv.drop(columns = ["id"], inplace = True)

In [ ]:
def cleaning(dataset):
    features = dataset.columns.tolist()
    dataset['mean_features'] = 0.1 * dataset[features].mean(axis=1)
    dataset['std_features'] = dataset[features].std(axis=1)
    dataset['max_features'] = dataset[features].max(axis=1)
    dataset['min_features'] = dataset[features].min(axis=1)
    dataset['median_features'] = 0.1 * dataset[features].median(axis=1)
    dataset['sum_features'] = dataset[features].sum(axis=1)
    dataset = dataset.drop(features, axis=1)
    return dataset

train_csv = cleaning(train_csv)
test_csv = cleaning(test_csv)

In [ ]:
train_csv = pd.concat([train_csv, Y], axis=1)

In [ ]:
train_csv.head()

/opt/conda/lib/python3.10/site-packages/IPython/core/formatters.py:344: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/conda/lib/python3.10/site-packages/IPython/core/formatters.py:344: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,mean_features,std_features,max_features,min_features,median_features,sum_features,FloodProbability
0,0.470,1.750188,8,2,0.45,94,0.445
1,0.470,2.296450,9,0,0.40,94,0.450
2,0.495,1.932411,8,1,0.50,99,0.530
3,0.520,1.641565,8,2,0.50,104,0.535
4,0.360,1.500877,6,1,0.30,72,0.415


## Definitions of light autoML

In [ ]:
N_THREADS = 4
N_FOLDS = 15
TEST_SIZE = 0.1
TIMEOUT = 1
RANDOM_STATE = 42
TARGET_NAME = "FloodProbability"

## Train Test Split

In [ ]:
train_data, val_data = train_test_split(train_csv,test_size=TEST_SIZE,random_state=RANDOM_STATE)

In [ ]:
task = Task('reg', loss='mse', metric='r2')

roles = {'target': TARGET_NAME}

In [ ]:
automl = TabularAutoML(
        task = task,
        timeout = TIMEOUT,
        cpu_limit = N_THREADS,
        general_params = {'use_algos':[['lgb_tuned', 'cb_tuned']]},
        selection_params={'mode' : 2},
        tuning_params = {'max_tuning_time': 3},
        reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE})

In [ ]:
train_predictions = automl.fit_predict(train_data,
                                       roles = {'target': TARGET_NAME},
                                       verbose = 1)

[18:28:24] Stdout logging level is INFO.
[18:28:24] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[18:28:24] Task: reg

[18:28:24] Start automl preset with listed constraints:
[18:28:24] - time: 1.00 seconds
[18:28:24] - CPU: 4 cores
[18:28:24] - memory: 16 GB

[18:28:24] Train data shape: (1006161, 7)



/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:37: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


[18:28:33] Layer 1 train process start. Time left -7.87 secs
[18:29:07] Selector_LightGBM fitting and predicting completed
[18:29:34] Selector_LightGBM fitting and predicting completed
[18:30:06] Selector_LightGBM fitting and predicting completed
[18:30:31] Selector_LightGBM fitting and predicting completed
[18:31:19] Selector_LightGBM fitting and predicting completed
[18:31:42] Selector_LightGBM fitting and predicting completed
[18:32:04] Selector_LightGBM fitting and predicting completed
[18:32:33] Selector_LightGBM fitting and predicting completed
[18:32:33] Start hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM ... Time budget is 1.00 secs
[18:33:05] Hyperparameters optimization for Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM completed
[18:33:05] Start fitting Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM ...
[18:33:39] Time limit exceeded after calculating fold 0

[18:33:39] Fitting Lvl_0_Pipe_0_Mod_0_Tuned_LightGBM finished. score = 0.8714547139245636
[18:33:39] Lvl_0_Pipe_0_Mod_0_Tu

In [ ]:
val_preds = automl.predict(val_data)#testing on val_data

print(f'OOF score: {r2_score(val_data["FloodProbability"].values, val_preds.data[:, 0])}')

OOF score: 0.8687340128885203


In [ ]:
print(pd.__version__)

2.2.2


In [ ]:
pip install --upgrade pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 49.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
fitter 1.7.0 requires joblib<2.0.0,>=1.3.1, but you have joblib 1.2.0 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
lightautoml 0.3.8.1 requires pandas<2.0.0, but you have pandas 2.2.2 which is incompatible.
momepy 0.7.0 requir

In [ ]:
import pandas as pd
print(pd.__version__)

2.2.2


In [ ]:
test_preds = automl.predict(test_csv)

In [ ]:
test_preds = pd.DataFrame(test_preds)
test_preds = test_preds[0]

In [ ]:
print(test_preds)

0         0.579876
1         0.455998
2         0.449682
3         0.466270
4         0.466240
            ...   
745300    0.469883
745301    0.443376
745302    0.620774
745303    0.548052
745304    0.528989
Name: 0, Length: 745305, dtype: float32


In [ ]:
subs = {'id': ids, 'FloodProbability': test_preds}
subs = pd.DataFrame(subs)

subs.to_csv("sub.csv")

AttributeError: 'Index' object has no attribute '_format_native_types'

In [ ]:
subs

NameError: name 'subs' is not defined